## Import

In [6]:
import random
import pandas as pd
import numpy as np
import os
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms as T
import albumentations

import albumentations as A
from albumentations.pytorch import transforms

import torchvision.models as models

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

In [7]:
from albumentations.pytorch.transforms import ToTensorV2

In [8]:
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')

In [9]:
torch.backends.mps.is_available()

True

## Hyperparameter Setting

In [10]:
CFG = {
    'IMG_SIZE':800,
    'IMG_SIZE_D':800,
    'EPOCHS':40,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':10,
    'SEED':41
}

## Fixed RandomSeed

In [11]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    torch.backends.mps.deterministic = True
    torch.backends.mps.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing
#### 1. Load Dataframe
#### 2. 결측치 보완
#### 3. Train / Validation Split
#### 4. Numeric Feature Scaling / Categorical Featrue Label-Encoding

In [12]:
train_df = pd.read_csv('/Users/krc/Documents/breast_dacon/open/train.csv')
test_df = pd.read_csv('/Users/krc/Documents/breast_dacon/open/test.csv')

In [13]:
train_df.rename(columns = {'나이':'age',
                           '수술연월일':'yr_mn_dt',
                           '진단명':'diag_cat',
                           '암의 위치':'location',
                           '암의 개수':'sng_mul',
                           '암의 장경':'size_mm',
                           'DCIS_or_LCIS_여부':'DCIS_or_LCIS'},inplace=True)

test_df.rename(columns = {'나이':'age',
                           '수술연월일':'yr_mn_dt',
                           '진단명':'diag_cat',
                           '암의 위치':'location',
                           '암의 개수':'sng_mul',
                           '암의 장경':'size_mm',
                           'DCIS_or_LCIS_여부':'DCIS_or_LCIS'},inplace=True)

In [17]:
train_df, val_df, train_labels, val_labels = train_test_split(
                                                    train_df.drop(columns=['N_category']), 
                                                    train_df['N_category'], 
                                                    test_size=0.1,  # 0.2 
                                                    random_state=CFG['SEED']
                                                )

In [18]:
def get_values(value):
    return value.values.reshape(-1, 1)

numeric_cols = ['age', 'size_mm', 'ER_Allred_score', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2_SISH_ratio']
ignore_cols = ['ID', 'img_path', 'mask_path', 'yr_mn_dt', 'N_category']

for col in train_df.columns:
    if col in ignore_cols:
        continue
    if col in numeric_cols:
        scaler = StandardScaler()
        train_df[col] = scaler.fit_transform(get_values(train_df[col]))
        val_df[col] = scaler.transform(get_values(val_df[col]))
        test_df[col] = scaler.transform(get_values(test_df[col]))
    else:
        le = LabelEncoder()
        train_df[col] = le.fit_transform(get_values(train_df[col]))
        val_df[col] = le.transform(get_values(val_df[col]))
        test_df[col] = le.transform(get_values(test_df[col]))

## CustomDataset

In [19]:
os.getcwd()

'/Users/krc/Documents/breast_dacon/dacon_bc_prediction'

In [20]:
base = '/Users/krc/Documents/breast_dacon/open'

In [21]:
base + train_df['img_path'].iloc[0][1:]

'/Users/krc/Documents/breast_dacon/open/train_imgs/BC_01_2853.png'

In [22]:
class CustomDataset(Dataset):
    def __init__(self, medical_df, labels, transforms=None):
        self.medical_df = medical_df
        self.transforms = transforms
        self.labels = labels
        
    def __getitem__(self, index):
        img_path = base + self.medical_df['img_path'].iloc[index][1:]
        # print(img_path)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
                
        if self.labels is not None:
            label = self.labels[index]
            return image,  label #tabular = 2nd
        else:
            return image #, tabular
          
    def __len__(self):
        return len(self.medical_df)

In [23]:
train_transforms = A.Compose([
                            A.HorizontalFlip(),
                            A.VerticalFlip(),
                            A.Rotate(limit=180, border_mode=cv2.BORDER_CONSTANT,p=0.3),
                            A.GaussNoise(p=0.5), #NOISE ADD 
                            A.Resize(CFG['IMG_SIZE_D'],CFG['IMG_SIZE']),
                            #밝기/대비 조정
                            A.augmentations.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.pytorch.transforms.ToTensorV2()
                            ])

test_transforms = A.Compose([
                            A.Resize(CFG['IMG_SIZE_D'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),

                            A.pytorch.transforms.ToTensorV2()
                            ])

In [24]:
train_dataset = CustomDataset(train_df, train_labels.values, train_transforms)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df, val_labels.values, test_transforms)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Architecture

In [26]:
class ImgFeatureExtractor(nn.Module):
    def __init__(self):
        super(ImgFeatureExtractor, self).__init__()
        self.backbone = models.resnet50(weights=True)
#         for i, param in enumerate(self.backbone.parameters()):
#               if i >139:
#                 param.requires_grad= True
#        self.embedding = nn.Linear(1000,512)
        
    def forward(self, x):
        x = self.backbone(x)

#        x = self.embedding(x)
        return x
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.backbone.parameters():
            param.require_grad = False
        for param in self.backbone.fc.parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.backbone.parameters():
            param.require_grad = True

In [27]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.img_feature_extractor = ImgFeatureExtractor()
        self.classifier = nn.Sequential(
            nn.Linear(in_features=1000, out_features=1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(),
            
            nn.Linear(in_features=1024, out_features=512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            
            nn.Linear(in_features=512, out_features=256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),

            nn.Linear(in_features=256, out_features=128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),

            nn.Linear(in_features=128, out_features=64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            
            nn.Linear(in_features=64, out_features=1),
            nn.Sigmoid(),
        )
        
    def forward(self, img): 
        img_feature = self.img_feature_extractor(img)
        output = self.classifier(img_feature) 
        return output

## Train

In [28]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.BCEWithLogitsLoss().to(device)
    
    best_score = 0
    best_epcoh = 0 
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        
        for img, label in tqdm(iter(train_loader)): 
            img = img.float().to(device)
            label = label.float().to(device)
            
            optimizer.zero_grad()
            
            model_pred = model(img)
            print(model_pred)
            
            loss = criterion(model_pred, label.reshape(-1,1))
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        val_loss, val_score = validation(model, criterion, val_loader, device)
        print(f'Epoch [{epoch}], Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}] Val Score : [{val_score:.5f}]')
        
        if scheduler is not None: #epoch 마다 업데이트
            scheduler.step(val_loss) #val_loss
        
        if best_score < val_score:
            best_epoch = epoch
            best_score = val_score
            best_model = model
            torch.save(best_model.state_dict(), f'./{epoch}_model_{best_score}.pth')
    
    return best_model

In [29]:
def validation(model, criterion, val_loader, device):
    model.eval()
    pred_labels = []
    true_labels = []
    val_loss = []
    threshold = 0.5
    with torch.no_grad():
        for img, label in tqdm(iter(val_loader)): 
            true_labels += label.tolist()
            
            img = img.float().to(device)
            label = label.float().to(device)
            
            model_pred = model(img)
            
            loss = criterion(model_pred, label.reshape(-1,1))
            
            val_loss.append(loss.item())
            
            model_pred = model_pred.squeeze(1).to('cpu')  
            pred_labels += model_pred.tolist()
    
    pred_labels = np.where(np.array(pred_labels) > threshold, 1, 0)
    val_score = metrics.f1_score(y_true=true_labels, y_pred=pred_labels, average='macro')
    return np.mean(val_loss), val_score

## WANDB setting

In [32]:
import wandb

wandb.init()

wandb: Currently logged in as: rumj. Use `wandb login --relogin` to force relogin


In [33]:
api = wandb.Api()

run = api.run("rumj/dacon_bc_prediction/78r04zfs")
run.config["size"] = 800
run.config["epochs"] = 30
run.config["initial_lr"] = 1e-4
run.config["batch_size"] = 5
run.config["scheduler"] = "CosineAnnealingLR"
history = run.scan_history(keys=["Loss"])
losses = [row["Loss"] for row in history]

run.update()

## Run!!

In [34]:
model = nn.DataParallel(ClassificationModel())
wandb.watch(model)
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer = optimizer, T_max = CFG['EPOCHS'], verbose = True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Adjusting learning rate of group 0 to 1.0000e-04.


  0%|          | 0/90 [00:00<?, ?it/s]

tensor([[0.4558],
        [0.3228],
        [0.4044],
        [0.4685],
        [0.3751],
        [0.4456],
        [0.5401],
        [0.3749],
        [0.2760],
        [0.3385]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.2932],
        [0.2942],
        [0.3464],
        [0.4634],
        [0.4953],
        [0.5664],
        [0.3652],
        [0.3007],
        [0.4683],
        [0.3780]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.4232],
        [0.4928],
        [0.3779],
        [0.5050],
        [0.4086],
        [0.3158],
        [0.3119],
        [0.4581],
        [0.2770],
        [0.3905]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.2749],
        [0.4679],
        [0.4495],
        [0.3032],
        [0.4989],
        [0.3490],
        [0.4566],
        [0.4627],
        [0.3418],
        [0.3902]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.3944],
        [0.4589],
        [0.5450],
        [0.3853],
        [0.3554],
        [0

tensor([[0.3976],
        [0.3918],
        [0.5206],
        [0.2836],
        [0.5684],
        [0.5994],
        [0.2013],
        [0.3200],
        [0.3806],
        [0.3044]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.2296],
        [0.4340],
        [0.1230],
        [0.3495],
        [0.3580],
        [0.3545],
        [0.6718],
        [0.5459],
        [0.3581],
        [0.5969]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.5514],
        [0.3317],
        [0.1764],
        [0.3804],
        [0.4956],
        [0.4490],
        [0.1867],
        [0.3766],
        [0.6468],
        [0.3866]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.5218],
        [0.1206],
        [0.4436],
        [0.2918],
        [0.4127],
        [0.5508],
        [0.5821],
        [0.3946],
        [0.2811],
        [0.4011]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.5101],
        [0.4246],
        [0.1959],
        [0.4789],
        [0.2448],
        [0

tensor([[0.4509],
        [0.3288],
        [0.7947],
        [0.2262],
        [0.2658],
        [0.4354],
        [0.1070],
        [0.3541],
        [0.6775],
        [0.3452]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.3302],
        [0.4408],
        [0.8000],
        [0.1183],
        [0.3647],
        [0.2502],
        [0.3609],
        [0.3479],
        [0.7280],
        [0.2415]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.1951],
        [0.5511],
        [0.1189],
        [0.4365],
        [0.2969],
        [0.2487],
        [0.2539],
        [0.4610],
        [0.8615],
        [0.5000]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.2415],
        [0.3518],
        [0.2154],
        [0.2245],
        [0.8496],
        [0.7830],
        [0.1043],
        [0.3567],
        [0.3781],
        [0.3966]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.1584],
        [0.5906],
        [0.2005],
        [0.4213],
        [0.5566],
        [0

...

Epoch [11], Train Loss : [0.65655] Val Loss : [0.64020] Val Score : [0.71820]
Epoch 0.64: adjusting learning rate of group 0 to 9.9937e-05.


  0%|          | 0/90 [00:00<?, ?it/s]

tensor([[0.6982],
        [0.9521],
        [0.0767],
        [0.0283],
        [0.1724],
        [0.1612],
        [0.9465],
        [0.4470],
        [0.0172],
        [0.0254]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.3220],
        [0.8311],
        [0.3170],
        [0.9635],
        [0.0168],
        [0.1905],
        [0.8743],
        [0.0597],
        [0.0269],
        [0.0272]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.9822],
        [0.0618],
        [0.4644],
        [0.3273],
        [0.8002],
        [0.0417],
        [0.6678],
        [0.0155],
        [0.1905],
        [0.0145]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.5223],
        [0.2950],
        [0.1510],
        [0.0148],
        [0.0898],
        [0.9799],
        [0.8392],
        [0.6327],
        [0.0130],
        [0.0407]], device='mps:0', grad_fn=<SigmoidBackward0>)
tensor([[0.0846],
        [0.0190],
        [0.0222],
        [0.3237],
        [0.9543],
        [0

NotImplementedError: The operator 'aten::histc' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: ERROR Error while calling W&B API: internal database error (<Response [500]>)
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: N

## Inference

In [38]:
model.state_dict(torch.load('best_model.pth'))

OrderedDict([('module.img_feature_extractor.backbone.conv1.weight',
              tensor([[[[ 1.1621e-02,  1.3460e-02, -1.7270e-02,  ..., -4.1604e-02,
                         -4.3836e-02, -7.1067e-02],
                        [ 2.8087e-03,  4.7682e-03,  1.3908e-02,  ...,  2.4000e-03,
                         -2.0863e-02, -3.8716e-02],
                        [ 2.0766e-02,  2.2789e-02,  1.4497e-02,  ...,  1.0222e-01,
                          6.1504e-02,  5.0362e-02],
                        ...,
                        [-2.5452e-03,  2.7358e-02, -1.1304e-02,  ..., -1.2676e-01,
                         -7.6393e-02,  7.2464e-03],
                        [ 2.3338e-03,  4.6975e-02,  6.1181e-02,  ...,  2.3610e-02,
                         -3.3929e-02, -1.6875e-02],
                        [-8.1019e-02, -3.2434e-02, -1.8621e-02,  ...,  3.4472e-02,
                          2.1279e-02,  3.9584e-04]],
              
                       [[-2.0022e-02,  1.0747e-02,  2.2347e-02,  ...,  5.4279

In [35]:
test_dataset = CustomDataset(test_df, None, test_transforms)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [36]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    threshold = 0.5
    
    with torch.no_grad():
        for img in tqdm(iter(test_loader)): # , tabular   = 2nd
            img = img.float().to(device)
            # tabular = tabular.float().to(device)
            
            model_pred = model(img)
            
            model_pred = model_pred.squeeze(1).to('cpu')
            
            preds += model_pred.tolist()
    
    preds = np.where(np.array(preds) > threshold, 1, 0)
    
    return preds

In [37]:
preds = inference(model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

## Submission

In [38]:
submit = pd.read_csv('/Users/krc/Documents/breast_dacon/open/sample_submission.csv')

In [39]:
submit['N_category'] = preds
submit.to_csv('./submit_1110_size800.csv', index=False)

In [60]:
os.getcwd()

'/Users/krc/Documents/breast_dacon/dacon_bc_prediction'